In [1]:
# essential imports
import pandas as pd
from sklearn.model_selection import train_test_split

# Load preprocessed dataset
df = pd.read_csv("preprocess_data.csv")

# Define features (X) and multiple targets (y)
X = df[['Season', 'Day_of_the_week', 'DHI', 'DNI', 'GHI', 'Wind_speed', 'Humidity', 'Temperature']]
y = df[['PV_production', 'Wind_production']]

# Split the data 70-30 for multi-output
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, shuffle=True
)

train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)
train_data.to_csv("train_multi_output.csv", index=False)
test_data.to_csv("test_multi_output.csv", index=False)

print("Train/Test data saved for multi-output regression.")
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")


Train/Test data saved for multi-output regression.
Train shape: (27215, 8), Test shape: (11664, 8)
